In [1]:
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash", convert_system_message_to_human=True)

In [8]:
## setting PINECONE API KEY nad PINECONE ENVIRONMENT in the environment

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_ENV"] = "us-east-1"

In [9]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index_name="smart-assistant", embedding=gemini_embeddings)

In [10]:

from langchain.chains import RetrievalQA  

qa = RetrievalQA.from_chain_type(  
    llm=model,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
qa.invoke("who won ipl 2025?")  

{'query': 'who won ipl 2025?',
 'result': 'Royal Challengers Bengaluru won the IPL in 2025.'}

In [11]:
qa.invoke("who played champions trophhie semi finals?")  

{'query': 'who played champions trophhie semi finals?',
 'result': 'The semi-final matches of the 2025 ICC Champions Trophy were:\n\n*   India (A1) vs Australia (B2)\n*   New Zealand (A2) vs South Africa (B1)'}

In [12]:
from langchain.chains import RetrievalQAWithSourcesChain  
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=model,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
qa_with_sources.invoke("who won ipl 2025?")


{'question': 'who won ipl 2025?',
 'answer': 'Royal Challengers Bengaluru won IPL 2025.\n',
 'sources': '/Users/yash/Library/mcqgen/app/Assistant/pdf_docs/ipl2025.pdf'}

In [13]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationBufferMemory

# 🧠 Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# ConversationalRetrievalChain using .from_llm
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=vectorstore.as_retriever(),
    memory=memory,
)

print(rag_chain)

memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history') verbose=False combine_docs_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1b

/var/folders/v3/hj2d2g150gld7k2cm6q9d9gh0000gn/T/ipykernel_68094/3842712392.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [18]:
print(rag_chain.invoke("who scored most runs in ipl 2025?")["answer"])

Sai Sudharsan of Gujarat Titans scored the most runs in IPL 2025 with 759 runs.


In [19]:
print(rag_chain.invoke("which award has he won?")["answer"])

Sai Sudharsan has won the following awards:

*   Fantasy player of the season
*   Orange Cap (most runs)
*   Emerging player of the season
*   Most fours


In [20]:
print(rag_chain.invoke("for which team does he play?")["answer"])

Sai Sudharsan plays for Gujarat Titans.


In [21]:
print(rag_chain.invoke("who is the caption of that team?")["answer"])

Shubman Gill is the captain of Gujarat Titans.


In [22]:
print(rag_chain.invoke("how many runs did hw scored?")["answer"])

Shubman Gill scored 90 runs in the match between Gujarat Titans and Kolkata Knight Riders, and 43 runs in the match between Mumbai Indians and Gujarat Titans.
